In [5]:
import yfinance as yf
import pandas as pd
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
import numpy as np

In [6]:
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

In [7]:
session = CachedLimiterSession(
    limiter=Limiter(
        RequestRate(2, Duration.SECOND * 5)
    ),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)
session.headers["User-agent"] = "my-program/1.0"

### Helper Functions


In [60]:
def convert_to_thousands(value):
    if isinstance(value, (int, float)):
        return f"{value/1000:.0f}"
    return value


def calculate_ratio(numerator, denominator):
    if (
        isinstance(numerator, (int, float))
        and isinstance(denominator, (int, float))
        and denominator != 0
    ):
        return f"{numerator / denominator:.2f}"
    return "No result"


def convert_to_percent(value):
    if isinstance(value, float):
        return f"{value * 100:.2f}%"
    return value


def get_latest_value(df, metric):
    if metric in df.index:
        for col in df.columns:
            value = df.at[metric, col]
            if not pd.isna(value):
                return value
    return "Not Found"

## Metric Calculation Functions


In [73]:
def calculate_nwc_change(balance_sheet):
    # NWC (most broad interpretation) = Current Assets - Current Liabilities
    # DeltaNWC = Prior Period NWC - Current Period NWC
    cur_nwc = get_latest_value(balance_sheet, "Working Capital")
    prior_balance = balance_sheet.iloc[:, 1:]
    prior_nwc = get_latest_value(prior_balance, "Working Capital")
    if prior_nwc - cur_nwc == 0:
        print(f"prior: {prior_nwc}, current: {cur_nwc}")
    return prior_nwc - cur_nwc


def _calculate_non_cash_wc_for_period(balance_sheet):
    ar = get_latest_value(balance_sheet, "Accounts Receivable")
    inv = get_latest_value(balance_sheet, "Inventory")
    ap = get_latest_value(balance_sheet, "Accounts Payable")
    return ar + inv - ap


def calculate_non_cash_wc(balance_sheet):
    current = _calculate_non_cash_wc_for_period(balance_sheet)
    prior_bs = balance_sheet.iloc[:, 1:]
    prior = _calculate_non_cash_wc_for_period(prior_bs)
    return prior - current


def calculate_ufcf(cash_flow, income_statement, balance_sheet):
    # Unlevered Free Cash Flow
    # EBIT*(1-tax rate) + D&A – ∆NWC – CAPEX
    nopat = get_latest_value(income_statement, "EBIT") * (
        1 - get_latest_value(income_statement, "Tax Rate For Calcs")
    )  # Net Operating Profit After Tax
    dep_am = get_latest_value(cash_flow, "Depreciation And Amortization")
    change_in_non_cash_wc = calculate_non_cash_wc(balance_sheet)
    capex = get_latest_value(cash_flow, "Capital Expenditure")
    # Calculate UFCF
    ufcf = nopat + dep_am - change_in_non_cash_wc - capex
    return ufcf


def calculate_sfcf(cash_flow):
    # Simple Free Cash Flow
    operating_cash = get_latest_value(cash_flow, "Operating Cash Flow")
    capex = get_latest_value(cash_flow, "Capital Expenditure")
    return operating_cash - capex


def calculate_net_net_wc_to_price(balance_sheet, market_cap):
    # ((Cash + Short Term Marketable Investments + (Accounts Receivable * 75%) + (Inventory * 50%)  - Total Liabilities)
    # / # of shares) / Price
    # convert /shares/price -> /market_cap
    ccesti = get_latest_value(
        balance_sheet, "Cash Cash Equivalents And Short Term Investments"
    )
    disc_AR = 0.75 * get_latest_value(balance_sheet, "Accounts Receivable")
    disc_INV = 0.5 * get_latest_value(balance_sheet, "Inventory")
    tot_liab = get_latest_value(balance_sheet, "Total Assets") - get_latest_value(
        balance_sheet, "Stockholders Equity"
    )
    return float(f"{(ccesti + disc_AR + disc_INV - tot_liab) / market_cap:.2f}")

## Main Fetcher Function


In [74]:
# NOTE: Need to adjust dictionary keys to match source sheet headers


def fetch_financial_data(ticker, session):
    stock = yf.Ticker(ticker, session)
    info = stock.info
    balance_sheet = stock.balance_sheet
    income_statement = stock.income_stmt
    cash_flow = stock.cashflow

    financial_data = {
        "Ticker": ticker,
        "Revenue (ttm)": convert_to_thousands(info.get("totalRevenue", "No result")),
        "Return on Assets (ttm)": convert_to_percent(
            info.get("returnOnAssets", "No result")
        ),
        "Return on Equity (ttm)": convert_to_percent(
            info.get("returnOnEquity", "No result")
        ),
        "EBITDA (s&p)": convert_to_thousands(info.get("ebitda", "No result")),
        "Enterprise Value": convert_to_thousands(
            info.get("enterpriseValue", "No result")
        ),
        "Dividend": info.get("dividendRate", "No result"),
        "CCESTI": convert_to_thousands(
            get_latest_value(
                balance_sheet, "Cash Cash Equivalents And Short Term Investments"
            )
        ),
        "Total Debt": convert_to_thousands(info.get("totalDebt", "No result")),
        "Earnings Per Share": info.get("trailingEps", "No result"),
        "Book Value": info.get(
            "bookValue", "No result"
        ),  # currently returns book value per share, not Assets - Liabilities (in other words, Equities)
        "Levered Free Cash Flow (ttm)": convert_to_thousands(
            info.get("freeCashflow", "No result")
        ),
        "Cash Flow from Operations (ttm)": convert_to_thousands(
            info.get("operatingCashflow", "No result")
        ),
        "Cash Flow from Investing (ttm)": convert_to_thousands(
            get_latest_value(cash_flow, "Investing Cash Flow")
        ),
        "Cash Flow from Financing (ttm)": convert_to_thousands(
            get_latest_value(cash_flow, "Financing Cash Flow")
        ),
        "Tangible Book Value": convert_to_thousands(
            get_latest_value(balance_sheet, "Tangible Book Value")
        ),
        "Pretax Income": convert_to_thousands(
            get_latest_value(income_statement, "Pretax Income")
        ),
        "Income Tax Expense": convert_to_thousands(
            get_latest_value(income_statement, "Pretax Income")
            * get_latest_value(income_statement, "Tax Rate For Calcs")
        ),
        "Interest Expense": convert_to_thousands(
            get_latest_value(income_statement, "Interest Expense")
        ),
        "EBIT": convert_to_thousands(get_latest_value(income_statement, "EBIT")),
        "Capital Expenditure": convert_to_thousands(
            get_latest_value(cash_flow, "Capital Expenditure")
        ),
        "Current Assets": convert_to_thousands(
            get_latest_value(balance_sheet, "Current Assets")
        ),
        "Cash": convert_to_thousands(info.get("totalCash")),
        "Short-Term Investments": convert_to_thousands(
            get_latest_value(balance_sheet, "Other Short Term Investments")
        ),
        "Current Inventory": convert_to_thousands(
            get_latest_value(balance_sheet, "Inventory")
        ),
        "Current Liabilities": convert_to_thousands(
            get_latest_value(balance_sheet, "Current Liabilities")
        ),
        "Net Receivables": convert_to_thousands(
            get_latest_value(balance_sheet, "Receivables")
        ),
        "Total Assets": convert_to_thousands(
            get_latest_value(balance_sheet, "Total Assets")
        ),
        "Total Liabilities": convert_to_thousands(
            get_latest_value(balance_sheet, "Total Assets")
            - get_latest_value(balance_sheet, "Stockholders Equity")
        ),
        "Retained Earnings": convert_to_thousands(
            get_latest_value(balance_sheet, "Retained Earnings")
        ),
        "∆NWC ttm": convert_to_thousands(calculate_nwc_change(balance_sheet)),
        "Working Capital": convert_to_thousands(
            get_latest_value(balance_sheet, "Working Capital")
        ),
        "Effective Tax Rate": get_latest_value(income_statement, "Tax Rate For Calcs"),
        "Tax Shield": convert_to_thousands(
            get_latest_value(income_statement, "Interest Expense")
            * get_latest_value(income_statement, "Tax Rate For Calcs")
        ),
        "Free Cash Flow": convert_to_thousands(info.get("freeCashflow", "No result")),
        "Unlevered Free Cash Flow": convert_to_thousands(
            calculate_ufcf(cash_flow, income_statement, balance_sheet)
        ),
        "Simple Free Cash Flow": convert_to_thousands(calculate_sfcf(cash_flow)),
        "Net Cash Flow": convert_to_thousands(
            get_latest_value(cash_flow, "Operating Cash Flow")
            + get_latest_value(cash_flow, "Investing Cash Flow")
            + get_latest_value(cash_flow, "Financing Cash Flow")
        ),
        "Net Cash": convert_to_thousands(
            info.get("totalCash", 0) - info.get("totalDebt", 0)
        ),
        "Z1": calculate_ratio(
            get_latest_value(balance_sheet, "Working Capital"),
            get_latest_value(balance_sheet, "Total Assets"),
        ),  # WC / Total Assets
        "Z2": calculate_ratio(
            get_latest_value(balance_sheet, "Retained Earnings"),
            get_latest_value(balance_sheet, "Total Assets"),
        ),  # RE / Total Assets
        "Z3": calculate_ratio(
            get_latest_value(income_statement, "EBIT"),
            get_latest_value(balance_sheet, "Total Assets"),
        ),  # EBIT / Total Assets
        "Z4": calculate_ratio(
            info.get("marketCap", 0),
            get_latest_value(balance_sheet, "Total Assets")
            - get_latest_value(balance_sheet, "Stockholders Equity"),
        ),  # Market Cap / Total Liabilities
        "Z5": calculate_ratio(
            get_latest_value(income_statement, "Total Revenue"),
            get_latest_value(balance_sheet, "Total Assets"),
        ),  # Rev / Total Assets
        "Sales to Price Ratio": calculate_ratio(
            info.get("totalRevenue"), info.get("marketCap")
        ),
        "EBITDA to Enterprise Value Ratio": calculate_ratio(
            1, info.get("enterpiseToEbitda")  # just invert readily available metric
        ),
        "Tangible Book Value to Price Ratio": calculate_ratio(
            get_latest_value(balance_sheet, "Net Tangible Assets"),
            info.get("marketCap"),
        ),
        "Book Value to Price Ratio": calculate_ratio(
            1, info.get("priceToBook")
        ),  # just invert the priceToBook that is available easily
        "Earnings to Price Ratio": calculate_ratio(
            info.get("trailingEps"), info.get("marketCap")
        ),
        "Cash Flow from Operations to Price Ratio": calculate_ratio(
            info.get("operatingCashflow"), info.get("marketCap")
        ),
        "Simple Free Cash Flow to Price Ratio": calculate_ratio(
            calculate_sfcf(cash_flow), info.get("marketCap")
        ),
        "Net Cash Flow to Price Ratio": calculate_ratio(
            get_latest_value(cash_flow, "Operating Cash Flow")
            + get_latest_value(cash_flow, "Investing Cash Flow")
            + get_latest_value(cash_flow, "Financing Cash Flow"),
            info.get("marketCap"),
        ),
        "Dividend to Price Ratio": calculate_ratio(
            info.get("dividendRate"), info.get("marketCap")
        ),
        "Cash to Price Ratio": calculate_ratio(
            info.get("totalCash"), info.get("marketCap")
        ),
        "Net Cash to Price Ratio": calculate_ratio(
            info.get("totalCash", 0) - info.get("totalDebt", 0), info.get("marketCap")
        ),
        "Net Net Working Capital to Price Ratio": calculate_net_net_wc_to_price(
            balance_sheet, info.get("marketCap")
        ),
        "Asset to Price Ratio": calculate_ratio(
            get_latest_value(balance_sheet, "Total Assets"), info.get("marketCap")
        ),
        "Retained Earnings to Market Capitalization": calculate_ratio(
            get_latest_value(balance_sheet, "Retained Earnings"), info.get("marketCap")
        ),
        "Revenue to Debt Ratio": calculate_ratio(
            info.get("totalRevenue"), info.get("totalDebt")
        ),
        "Market Capitalization to Enterprise Value Ratio": calculate_ratio(
            info.get("marketCap"), info.get("enterpriseValue")
        ),
    }

    return financial_data

### Wrapper Function


In [75]:
def get_ticker_info(ticker, session):
    financial_data = fetch_financial_data(ticker, session)
    df = pd.DataFrame([financial_data])
    return df

In [76]:
gme = get_ticker_info("GME", session)

In [77]:
gme.head()

,Ticker,Revenue (ttm),Return on Assets (ttm),Return on Equity (ttm),EBITDA (s&p),Enterprise Value,Dividend,CCESTI,Total Debt,Earnings Per Share,...,Simple Free Cash Flow to Price Ratio,Net Cash Flow to Price Ratio,Dividend to Price Ratio,Cash to Price Ratio,Net Cash to Price Ratio,Net Net Working Capital to Price Ratio,Asset to Price Ratio,Retained Earnings to Market Capitalization,Revenue to Debt Ratio,Market Capitalization to Enterprise Value Ratio
0,GME,4917500,-0.78%,1.93%,23800,7101297,No result,1199300,588700,0.08,...,-0.02,-0.02,No result,0.10,0.05,0.02,0.25,-0.02,8.35,1.51


In [66]:
gme.to_csv("gme.csv")

## Getting it to sheets


In [ ]:
# Load in current metrics sheet
#

## SANDBOX


In [26]:
gme["Net Net Working Capital to Price Ratio"]

0    0.019928
Name: Net Net Working Capital to Price Ratio, dtype: float64

In [43]:
GME = yf.Ticker("GME", session=session)
cash_flow = GME.cash_flow
balance_statement = GME.balance_sheet
inc_stmt = GME.income_stmt
info = GME.info

In [56]:
def calculate_nwc_change(balance_sheet):
    # NWC (most broad interpretation) = Current Assets - Current Liabilities
    # DeltaNWC = Prior Period NWC - Current Period NWC
    cur_nwc = get_latest_value(balance_sheet, "Working Capital")
    prior_balance = balance_sheet.iloc[:, 1:]
    prior_nwc = get_latest_value(prior_balance, "Working Capital")
    if prior_nwc - cur_nwc == 0:
        print(f"prior: {prior_nwc}, current: {cur_nwc}")
    return prior_nwc - cur_nwc

In [57]:
calculate_nwc_change(balance_statement)

-55100000.0

In [69]:
get_latest_value(inc_stmt, "EBIT")

-29700000.0

In [32]:
GME.info.get("bookValue") / (
    GME.info.get("marketCap") / GME.info.get("sharesOutstanding")
)

0.1401771027997391

In [33]:
1 / GME.info.get("priceToBook")

0.14017711659035426

In [35]:
GME.info.get("trailingEps") / GME.info.get("marketCap")

7.470605498193525e-12

In [71]:
GME.info.get("ebitda")

23800000

In [72]:
get_latest_value(balance_statement, "Total Assets") - get_latest_value(
    balance_statement, "Stockholders Equity"
)

1370400000.0

In [147]:
def get_latest_value(df, metric):
    if metric in df[df.columns[0]]:
        i = 0
        for i in range(0, df.shape[1]):
            if not np.isnan(df[df.columns[i]][metric]):
                return df[df.columns[i]][metric]
    return "Not Found"

In [148]:
get_latest_value(cash_flow, "Investing Cash Flow")

-33200000.0

In [149]:
get_latest_value(cash_flow, "Financing Cash Flow")

-11600000.0

In [150]:
get_latest_value(cash_flow, "Net Other Financing Charges")

-3000000.0

In [ ]:
ncf2p = calculate_ratio(
    int(info.get("operatingCashflow"))
    + int(get_latest_value(cash_flow, "Investing Cash Flow"))
    + int(get_latest_value(cash_flow, "Financing Cash Flow")),
    int(info.get("marketCap")),
)
ncf2p

In [112]:
int(get_latest_value(cash_flow, "Investing Cash Flow"))

-238600000

In [104]:
info.get("operatingCashflow")

-210800000

In [99]:
cash_flow

,2024-01-31,2023-01-31,2022-01-31,2021-01-31,2020-01-31
Free Cash Flow,-238600000.0,52300000.0,-496300000.0,63700000.0,NaN
Repurchase Of Capital Stock,NaN,NaN,0.0,0.0,-198700000.0
Repayment Of Debt,-10700000.0,-3900000.0,-332400000.0,-255300000.0,NaN
Issuance Of Debt,NaN,0.0,0.0,197100000.0,0.0
Issuance Of Capital Stock,0.0,0.0,1672800000.0,0.0,NaN
...,...,...,...,...,...
Depreciation,55300000.0,60300000.0,73600000.0,NaN,NaN
Operating Gains Losses,1500000.0,-4700000.0,23600000.0,-28800000.0,NaN
Gain Loss On Sale Of PPE,1500000.0,-4700000.0,5400000.0,-27300000.0,NaN
Gain Loss On Sale Of Business,NaN,NaN,0.0,0.0,9100000.0


In [98]:
get_latest_value(cash_flow, "Operating Cash Flow")

'No result'

In [27]:
4_917_499 * 1000

4917499000

In [31]:
4_917_499_000

4917499000

In [151]:
aapl = yf.Ticker("AAPL")

In [152]:
aapl.income_stmt

,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Tax Effect Of Unusual Items,0.0,0.0,0.0,0.0
Tax Rate For Calcs,0.147,0.162,0.133,0.144
Normalized EBITDA,129188000000.0,133138000000.0,123136000000.0,81020000000.0
Net Income From Continuing Operation Net Minority Interest,96995000000.0,99803000000.0,94680000000.0,57411000000.0
Reconciled Depreciation,11519000000.0,11104000000.0,11284000000.0,11056000000.0
Reconciled Cost Of Revenue,214137000000.0,223546000000.0,212981000000.0,169559000000.0
EBITDA,129188000000.0,133138000000.0,123136000000.0,81020000000.0
EBIT,117669000000.0,122034000000.0,111852000000.0,69964000000.0
Net Interest Income,-183000000.0,-106000000.0,198000000.0,890000000.0
Interest Expense,3933000000.0,2931000000.0,2645000000.0,2873000000.0
